In [1]:
from io import BytesIO
import zipfile

Now, we have to load the zip file from our google drive and unzip it for usage.

In [2]:
zip_ref = zipfile.ZipFile(r'C:\Users\jaswa\Dropbox\PC\Desktop\ASL Recognition\archive(1).zip', mode = 'r')
print(zip_ref.namelist())
zip_ref.extractall("/tmp")
zip_ref.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Now that we have the data, we can proceed to clean it up and prepare it for training and validation.

First, let us import all the required modules

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils, callbacks
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
gen = ImageDataGenerator(rescale=1./255,
                         rotation_range=10,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         brightness_range=[0.2, 1.0],
                         zoom_range=0.2,
                         fill_mode="nearest",
                         validation_split=0.1) 
train = gen.flow_from_directory("/tmp/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="training",batch_size=256)
val = gen.flow_from_directory("/tmp/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="validation",batch_size=256)

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [7]:
model = tf.keras.Sequential()
model.add(VGG16(weights="imagenet",input_shape=(64,64,3),include_top=False))
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU(0.2))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU(0.2))
model.add(Dropout(0.3))
model.add(Dense(29,activation="softmax"))

In [8]:
opt = Adam(0.00001)
loss = CategoricalCrossentropy()

In [9]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [10]:
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", 
                                        patience=5, restore_best_weights = True)

In [11]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:
history = model.fit(train,validation_data = val,epochs = 11,shuffle = True,verbose = 1, callbacks = [earlystopping])

Epoch 1/11
 89/306 [=======>......................] - ETA: 2:18 - loss: 3.2450 - accuracy: 0.1422

Now, let us try predicting some alphabets from images taken off the internet.

In [ ]:
test = ImageDataGenerator(rescale=1./255).flow_from_directory("/tmp/asl_alphabet_test/",
                                    target_size=(64, 64), class_mode= 'categorical', shuffle = False)

test.class_indices.keys()

In [ ]:
res = model.evaluate(test, verbose = 0)
pred = model.predict(test)
pred = np.argmax(pred)

#mapping the labels
labels = (train.class_indices)
lables = dict((v, k) for k, v in labels.items())
pred = [lables[k] for k in pred]
print([el for el in pred])

In [ ]:
pred = model.predict(test)

for i in pred:
  print(np.argmax(i))

In [ ]:
model.save('model1.h5')